In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [24]:
from torchvision import models, transforms
from torch.autograd import Variable
from PIL import Image
import numpy as np
from path import Path as path
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
import time
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from Easy_Image import detect

d:\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")
d:\Anaconda3\lib\site-packages\torchvision\models\squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
d:\Anaconda3\lib\site-packages\torchvision\models\squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


In [4]:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Scale(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

d:\Anaconda3\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [5]:
imagenet = models.squeezenet1_1(pretrained=True)

d:\Anaconda3\lib\site-packages\torchvision\models\squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
d:\Anaconda3\lib\site-packages\torchvision\models\squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


In [12]:

def PIL_to_raw(img_pil):
    img_tensor = preprocess(img_pil)
    img_tensor.unsqueeze_(0)
    img_variable = Variable(img_tensor)
    fc_out = imagenet(img_variable)
    return fc_out.data.numpy()

def pred_vector(img_pil):
    raw = PIL_to_raw(img_pil)
    preds = raw[0]
    result = np.array([np.exp(p) for p in preds])
    return result / sum(result)

def cv2_to_PIL(cv2_im):
    tmp = cv2.cvtColor(cv2_im,cv2.COLOR_BGR2RGB)
    pil_im = Image.fromarray(tmp)
    return pil_im

In [7]:
mydata = "set"

In [13]:
t0 = time.time()
X, Y = [], []
for d in path(mydata).dirs():
    for f in d.files():
        Y.append(d.name)
        X.append(pred_vector(cv2_to_PIL(cv2.imread(f))))
t1 = time.time() - t0

In [14]:
t1

5.2290754318237305

In [18]:
t2 = time.time()
Y2 = pd.get_dummies(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y2, test_size=0.2)
rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
rf.fit(X_train,y_train)
t3 = time.time() - t2

In [19]:
t3

0.31919050216674805

In [20]:
y_pred = rf.predict(X_test)

In [22]:
f1_score(y_test, y_pred, average='micro')

0.7777777777777778

In [23]:
accuracy_score(y_test, y_pred)

0.7777777777777778

In [25]:
t5 = time.time()
X = []
for d in path(mydata).dirs():
    for f in d.files():
        img = detect.EasyImageFile(f)
        X.append(img.describe())
t6 = time.time() - t5

In [26]:
t6

2.709021806716919

In [27]:
t7 = time.time()
Y2 = pd.get_dummies(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y2, test_size=0.2)
rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
rf.fit(X_train,y_train)
t8 = time.time() - t7

In [28]:
t8

0.14652800559997559

In [29]:
y_pred = rf.predict(X_test)

In [30]:
f1_score(y_test, y_pred, average='micro')

0.8888888888888888

In [31]:
accuracy_score(y_test, y_pred)

0.8888888888888888

In [50]:
t9 = time.time()
X = []
for d in path(mydata).dirs():
    for f in d.files():
        img = detect.EasyImageFile(f)
        X.append(img.signature())
t10 = time.time() - t9

In [51]:
t10

1.5419697761535645

In [52]:
t11 = time.time()
Y2 = pd.get_dummies(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y2, test_size=0.2)
rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
rf.fit(X_train,y_train)
t12 = time.time() - t11

In [53]:
t12

0.1378648281097412

In [54]:
y_pred = rf.predict(X_test)

In [55]:
f1_score(y_test, y_pred, average='micro')

0.6666666666666666

In [56]:
accuracy_score(y_test, y_pred)

0.6666666666666666

In [57]:
t13 = time.time()
X = []
for d in path(mydata).dirs():
    for f in d.files():
        img = detect.EasyImageFile(f)
        sig = img.signature()
        pv = pred_vector(cv2_to_PIL(img.getimg()))
        des = img.describe()
        X.append(np.hstack((sig, pv, des)))
t14 = time.time() - t13

In [58]:
t15 = time.time()
Y2 = pd.get_dummies(Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y2, test_size=0.2)
rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
rf.fit(X_train,y_train)
t16 = time.time() - t15

In [62]:
t16

0.13979601860046387

In [59]:
y_pred = rf.predict(X_test)

In [60]:
f1_score(y_test, y_pred, average='micro')

0.8888888888888888

In [61]:
accuracy_score(y_test, y_pred)

0.8888888888888888